In [2]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import load_dataset, Dataset
import json
import re
from hanspell import spell_checker
from datasets import Dataset

c:\Users\leekm\anaconda3\envs\Pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# ===== 1. 데이터 로드 =====
with open("prompt_dataset_input_label22.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# ===== 1.5 정제 함수 정의 =====
def clean_text(text, use_spellcheck=False):
    text = re.sub(r"[^\uAC00-\uD7A3a-zA-Z0-9\s.,!?]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    if use_spellcheck:
        try:
            text = spell_checker.check(text).checked
        except:
            pass
    return text

def clean_raw_data(data, min_len=5, max_len=300, use_spellcheck=False):
    seen = set()
    cleaned = []
    for item in data:
        inp = clean_text(item["input"], use_spellcheck)
        tgt = clean_text(item["label"], use_spellcheck)
        if min_len <= len(inp) <= max_len and inp not in seen:
            cleaned.append({"input": inp, "label": tgt})
            seen.add(inp)
    return cleaned

# ===== 1.6 데이터 정제 적용 =====
cleaned_data = clean_raw_data(raw_data, use_spellcheck=False)  # 맞춤법 검사 True면 느려짐

# ===== 2. Dataset 변환 및 분할 =====
raw_dataset = Dataset.from_list(cleaned_data)
dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_dir = "./t5_finetuned_result"  # ✅ 저장된 디렉토리명 정확히 지정

tokenizer = T5Tokenizer.from_pretrained(model_dir)
tokenizer.pad_token = tokenizer.eos_token  # 꼭 지정!

model = T5ForConditionalGeneration.from_pretrained(model_dir).to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def preprocess_batch(examples):
    inputs = tokenizer(
        examples["input"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["label"],
            padding="max_length",
            truncation=True,
            max_length=128
        )["input_ids"]
    labels = [
        [token if token != tokenizer.pad_token_id else -100 for token in label]
        for label in labels
    ]
    inputs["labels"] = labels
    return inputs

train_dataset = train_dataset.map(preprocess_batch, batched=True, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(preprocess_batch, batched=True, remove_columns=eval_dataset.column_names)

train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/2475 [00:00<?, ? examples/s]c:\Users\leekm\anaconda3\envs\Pytorch\lib\site-packages\transformers\tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 276/276 [00:00<00:00, 4919.08 examples/s]


In [ ]:
training_args = TrainingArguments(
    output_dir="./ke-t5-base_finetuned",
    evaluation_strategy="epoch",
    logging_strategy="epoch",            # ✅ 로그도 epoch마다 출력
    save_strategy="epoch",
    save_total_limit=2,

    learning_rate=5e-5,
    lr_scheduler_type="cosine",

    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,

    num_train_epochs=20,
    warmup_ratio=0.1,
    weight_decay=0.01,

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    report_to="none"
)


In [ ]:
# 6. Trainer 정의 및 학습 (EarlyStopping 추가)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

In [25]:
import re

# 🔧 출력 후처리 함수
def clean_output(text):
    text = text.replace("<pad>", "").replace("</s>", "")
    text = re.sub(r"\b(\w+)(\s+\1)+\b", r"\1", text)  # 반복 단어 제거
    text = re.sub(r"\s{2,}", " ", text)  # 중복 공백 제거
    sentences = re.split(r'(?<=[.?!])\s+|\n+', text.strip())
    return sentences[0].strip() if sentences else text.strip()

# 🔍 프롬프트 리라이팅 함수 (beam search 기반)
def refine_prompt(bad_prompt, max_new_tokens=64):
    prompt = (
        "다음은 사용자의 모호하거나 짧은 프롬프트입니다.\n"
        "이 프롬프트를 더 구체적이고 명확하게 바꿔주세요.\n\n"
        "예시1:\n"
        "프롬프트: 요약해줘\n"
        "개선된 프롬프트: 아래 기사 내용을 3줄 이내로 핵심만 요약해줘.\n\n"
        "예시2:\n"
        "프롬프트: 좋은 선물 추천해줘\n"
        "개선된 프롬프트: 20대 직장인 남자 친구에게 어울리는 생일 선물 3가지를 추천해줘.\n\n"
        "예시3:\n"
        "프롬프트: 스마트폰 추천\n"
        "개선된 프롬프트: 2024년 상반기 기준, 30대 여성 직장인에게 적합한 가성비 스마트폰 3가지를 추천해줘.\n\n"
        f"프롬프트: {bad_prompt}\n"
        "개선된 프롬프트:"
    )

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to(model.device)
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    do_sample=True,
    num_beams=4,
    top_p=0.8,
    temperature=0.9,
    repetition_penalty=1.3
)


    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return clean_output(result)


In [26]:
prompts = [
    "AI가 뭐야?",
    "요약해줘",
    "설명해줘",
    "좋은 선물물 추천",
    "전시회"
]

for p in prompts:
    result = refine_prompt(p)
    print(f"📝 입력: {p}")
    print(f"✅ 출력: {result}")
    print("="*60)


📝 입력: AI가 뭐야?
✅ 출력: 소비자 리뷰에서 배송, 품질, CS 항목 모두 좋은 평가를 받은 쇼핑몰 3곳을 소개해줘.
📝 입력: 요약해줘
✅ 출력: 실제 구매 데이터를 기반으로 판매가 높은 상품 3가지를 추천해줘.
📝 입력: 설명해줘
✅ 출력: 자취생이 가장 많이 찾는 국내 유명 브랜드 3곳을 소개하고 인기 이유를 알려줘.
📝 입력: 좋은 선물물 추천
✅ 출력: 실제 구매 데이터를 기반으로 판매가 높은 제품 3가지를 추천해줘.
📝 입력: 전시회
✅ 출력: 모바일 쇼핑 앱 내 베스트셀러 상품 3가지를 소개해줘.


In [27]:
# 타이밍 포함 label 예시 출력
for d in cleaned_data:
    if "타이밍" in d["label"]:
        print(d["label"])

이직 시 연봉 협상 전략과 말해야 할 타이밍을 알려줘.
경력직 연봉 협상에서 유리하게 협상하는 법과 타이밍을 알려줘.
이직 시 연봉 협상 전략과 말해야 할 타이밍을 알려줘.
경력직 연봉 협상에서 유리하게 협상하는 법과 타이밍을 알려줘.
이직 시 연봉 협상 전략과 말해야 할 타이밍을 자세히 알려줘.
경력직 연봉 협상에서 유리하게 협상하는 법과 타이밍을 자세히 알려줘.
